In [73]:
from my_functions import *

In [74]:
file= uproot.open("/home/ruben/Escritorio/BachelorThesisRuben/Data/Muon_trees.root") #opening the Root file with Uproot 

MuonTree_ZeroBias=file["MuonTree_ZeroBias;1"]
tree=file["MuonTree_ZeroBias;1"] #here we assign one of the keys to a variable 
tree.show() #by saying .show we display the list below, where we can see the datasets contained by "

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
muon_pt              | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
muon_eta             | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
muon_phi             | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
muon_e               | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
muon_type            | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
muon_charge          | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
muon_author          | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
muon_quality         | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
muon_passedIDCuts    | std::vector<bool>        | AsJagged(AsDtype('bool'), h...
jTower_et            | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
jTower_et_MeV        | std::

In [75]:
eta=MuonTree_ZeroBias.arrays(["LVL1Muon_eta"], entry_start=996, entry_stop=997)
phi=MuonTree_ZeroBias.arrays(["LVL1Muon_phi"], entry_start=996, entry_stop=997)
et=MuonTree_ZeroBias.arrays(["LVL1Muon_et"], entry_start=996, entry_stop=997)

In [76]:
#Remove field information and leave only the numeric value
eta=eta[eta.fields[0]]
phi=phi[phi.fields[0]]
et=et[et.fields[0]]

In [77]:
#jTower selector allows to select one event from the tower. We choose 996 
jTower_eta=jTower_selector(MuonTree_ZeroBias,"jTower_eta",996)
muon_eta=eta

jTower_phi=jTower_selector(MuonTree_ZeroBias,"jTower_phi",996)
muon_phi=phi

jTower_et=jTower_selector(MuonTree_ZeroBias,"jTower_et_MeV",996)
muon_et=et #Taking the massless approximation for muons: e_t = p_t (thats a good approximation)

In [78]:
#The idea is:

#First create an isolation function: 
    #-Compute the values of dr that are below a certain threshold and generate a boolean mask of Trues and False
    #-Apply the mask to the energy vector and sum it up. That'll give an estimate of the muon energy

#Effectively scale this for N events

In [79]:
def dr_threshold_boolean_mask_event(event_dr,threshold):

    """"
    Takes an array containing the delta r values for an event and its respective jTower,
    then it starts to iterate over al muons in the event and generates a boolean mask containing 'True'
    for all delta_r values that are below a certain threshold.

    By using NumPy we're able to do this in a vectorized way (meaning: operating over all the array at once, instead of looping), which is
    shorter and apparently more resource efficient

    The idea is to apply this boolean mask to the energy vector, that will select the energy elements that are associated with the muon
    """
    event_dr = np.array(event_dr)  # ensures it's a NumPy array
    return event_dr < threshold

In [80]:
#Now we want to generate pairs of [muon,jTower1],[muon,jTower2]...
#So we have an array where the left column is the value of a muon (always the same) and the right column contains all the jTower values
#associated with such muon

jTower_muon_pair_eta=jTower_muon_pair_maker(MuonTree_ZeroBias, "jTower_eta",996,muon_eta)
jTower_muon_pair_phi=jTower_muon_pair_maker(MuonTree_ZeroBias, "jTower_phi",996,muon_phi)

In [81]:
#Now compute the dr:

dr_jTower_muon=delta_r(jTower_muon_pair_eta,jTower_muon_pair_phi)
dr_jTower_muon

array([2.5315933 , 2.497702  , 2.3491788 , ..., 1.1259365 , 0.98580396,
       1.0574416 ], shape=(7680,), dtype=float32)

In [82]:
#Create the boolean mask

mask=dr_threshold_boolean_mask_event(dr_jTower_muon,0.4)

#Apply it to the energy vector

result=jTower_et[mask]

#And now sum the results. Take into account that there are negative values and im not sure if they should be considered
total=sum(result)
total #This is the energy recorded

np.float32(7050.0)